In [ ]:
#!pip3 install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index pypdf python-dotenv

In [ ]:
import logging
import sys
import torch
from pprint import pprint
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.prompts.prompts import SimpleInputPrompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
#!pip install huggingface_hub
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token('')


In [ ]:
documents = SimpleDirectoryReader(input_files=["data.pdf"]).load_data()
# import argparse
# parser = argparse.ArgumentParser(description='Process a file with SimpleDirectoryReader.')

# parser.add_argument('file', type=str)

# args = parser.parse_args()

# file_path = args.file
#documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

In [ ]:
system_prompt = "You are a data extractor. Extract the exact data from given document. Make sure to give answers as precise as possible. If no information found please reply 'no information available'"
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:

context_window = 4026 #@param
temperature = 0.0 #@param
model_name = 'meta-llama/Llama-2-7b-chat-hf' #@param

In [ ]:
llm = HuggingFaceLLM(
    context_window=context_window,
    max_new_tokens=256,
    generate_kwargs={"temperature": temperature, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True, "use_auth_token":True},

)

In [ ]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
user_queries = ['ORD Number?','Patient Sex?' ,'Patient Identifier?','Patient Occupation?','Patient Ethnicity?',"Patient's Sex?",'Patient Age?']

In [ ]:
answer = dict()
for i, user_query in enumerate(user_queries):
    query_engine = index.as_query_engine()
    response = query_engine.query(user_query)
    answer.update({user_query: response.response.strip()})

In [ ]:
pprint(answer)